In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy.stats
import networkx as nx
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from matplotlib.ticker import FormatStrFormatter
from sklearn.neighbors import radius_neighbors_graph
import sklearn.neighbors
from scipy.spatial import distance
from Bio import SeqIO
from Bio.KEGG import REST
from Bio.KEGG.KGML import KGML_parser
import io



# locals
import utils as ut


In [2]:
fPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/LR/LewisLabUSCS/Mouse/Mouse-2020-Baccin-LR-pairs.xlsx"

lr = pd.read_excel(fPath)
print(lr.shape)

lr['ligand'] = lr['Ligand.Mouse'].str.upper()
lr['receptor'] = lr['Receptor.Mouse'].str.upper()
lr.head()

(2000, 8)


,Pair.Name,Ligand.Mouse,Receptor.Mouse,Source,ManualAnnotation,Ligand.CC,Ligand.GO,Reference (PMID / KEGG),ligand,receptor
0,A2m-Lrp1,A2m,Lrp1,Ramilowski,Correct,Secreted,Other,"1702392,10652313, 12194978",A2M,LRP1
1,Adgre5-Cd55,Adgre5,Cd55,Baccin,Correct,Membrane,Other,23447688,ADGRE5,CD55
2,Adipoq-Adipor1,Adipoq,Adipor1,Ramilowski,Correct,Secreted,Other,12802337,ADIPOQ,ADIPOR1
3,Adipoq-Adipor2,Adipoq,Adipor2,Ramilowski,Correct,Secreted,Other,"12802337, 12802330",ADIPOQ,ADIPOR2
4,Adm-Calcrl&Ramp2,Adm,Calcrl&Ramp2,Baccin,Correct,Secreted,Other,10342881,ADM,CALCRL&RAMP2


In [3]:
sptDir =  "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDInputs/"

spt = {}

for f in os.listdir(sptDir):
    if "spt" in f and "macrophage" in f:
        print(f)
        fullPath = f"{sptDir}{f}"
        key = f.split("_")[0]
        sdf = pd.read_csv(fullPath)
        sdf = sdf.rename(columns={'Unnamed: 0' : 'gene'})
        sdf = sdf.set_index('gene')
        sdf = sdf.T
        sdf.index = sdf.index.str.replace("-", ".")
        spt[key] = sdf
    
print('done')

HFD8_macrophage_spt.csv
HFD14_macrophage_spt.csv
ND_macrophage_spt.csv
done


In [4]:
ligPath = f"/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/LRspec_ligand.csv"
recPath = f"/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/LRspec_receptor.csv"

lig = pd.read_csv(ligPath)
rec = pd.read_csv(recPath)

df = pd.concat([lig, rec], ignore_index=True)

print(f"{lig.shape=}")
print(f"{rec.shape=}")
print(df.head())

lig.shape=(18172, 15)
rec.shape=(17996, 15)
  key cellType      gene    type  true_positive  true_negative  \
0  ND  B cells      HRAS  ligand             53            703   
1  ND  B cells        CP  ligand             10            790   
2  ND  B cells  HSP90AA1  ligand            291            192   
3  ND  B cells   NECTIN3  ligand              1            895   
4  ND  B cells       KDR  ligand              1            885   

   false_positive  false_negative  sensitivity  specificity  precision  \
0             196             309     0.146409     0.781980   0.212851   
1             109             352     0.027624     0.878754   0.084034   
2             707              71     0.803867     0.213571   0.291583   
3               4             361     0.002762     0.995551   0.200000   
4              14             361     0.002762     0.984427   0.066667   

         F1  meanGroup  meanOther       LFC  
0  0.173486   0.204420   0.307008 -0.586740  
1  0.041580   0.055249

In [5]:
def parseKEGG(pathId):
    genes = []
    results = REST.kegg_get(pathId).read()
    current_section = None
    for line in results.rstrip().split("\n"):
        section = line[:12].strip()  # section names are within 12 columns
        if not section == "":
            current_section = section

        if current_section == "GENE":
            linesplit = line[12:].split("; ")
            gene_identifiers = linesplit[0]
            gene_id, gene_symbol = gene_identifiers.split()
    
            if not gene_symbol in genes:
                genes.append(gene_symbol)
    return genes

def getPathname(pathId):
    """A function to return the legg pathname"""
    result = REST.kegg_list(pathId).read()
    return result.split("\t")[1].split("-")[0].strip()
    

pathId = '04931'
pathId = f"mmu{pathId}"
genes = parseKEGG(pathId)
pathname = getPathname(pathId)
print(f"{pathname} has {len(genes)} genes")

genes = [x.upper() for x in genes]

mask = ((lr['ligand'].isin(genes)) | (lr['receptor'].isin(genes)))
ldf = lr[mask].reset_index()
print(f"number of lr pairs found in {pathname} {ldf.shape[0]}")
ldf.head()

Insulin resistance has 110 genes
number of lr pairs found in Insulin resistance 28


,index,Pair.Name,Ligand.Mouse,Receptor.Mouse,Source,ManualAnnotation,Ligand.CC,Ligand.GO,Reference (PMID / KEGG),ligand,receptor
0,119,Col1a1-Cd36,Col1a1,Cd36,Ramilowski,Correct,ECM,Other,2468670,COL1A1,CD36
1,127,Col1a2-Cd36,Col1a2,Cd36,Ramilowski,Correct,ECM,Other,10772928,COL1A2,CD36
2,327,Igf1-Insr,Igf1,Insr,Baccin,Correct,Secreted,GrowthFactor,23518924,IGF1,INSR
3,329,Igf2-Insr,Igf2,Insr,Ramilowski,Correct,Secreted,GrowthFactor,"9281335, 9722981",IGF2,INSR
4,348,Il6-Il6ra,Il6,Il6ra,Ramilowski,Correct,Secreted,Interleukin,"3136546,14557255, 12829785",IL6,IL6RA


In [6]:
print(genes)

['INS2', 'INS1', 'RPS6KA3', 'RPS6KA1', 'RPS6KA2', 'RPS6KA6', 'PPP1CA', 'PPP1CB', 'PPP1CC', 'PPP1CCB', 'PPP1R3A', 'PPP1R3C', 'PPP1R3B', 'PPP1R3D', 'PPP1R3E', 'INSR', 'IRS1', 'PTPN1', 'PTPN11', 'PTPRF', 'PIK3CA', 'PIK3CD', 'PIK3CB', 'PIK3R2', 'PIK3R1', 'PIK3R3', 'AKT1', 'AKT2', 'AKT3', 'MTOR', 'RPS6KB2', 'RPS6KB1', 'PRKCB', 'IL6', 'STAT3', 'SOCS3', 'GSK3B', 'GYS1', 'GYS2', 'TNF', 'TNFRSF1A', 'MAPK8', 'MAPK9', 'MAPK10', 'IKBKB', 'NFKBIA', 'NFKB1', 'RELA', 'NOS3', 'PRKCZ', 'TBC1D4', 'SLC2A4', 'PRKAA1', 'PRKAA2', 'PRKAB1', 'PRKAB2', 'PRKAG1', 'PRKAG3', 'PRKAG2', 'ACACB', 'CPT1B', 'AGT', 'SLC27A1', 'SLC27A4', 'SLC27A2', 'SLC27A3', 'SLC27A5', 'SLC27A6', 'CD36', 'PRKCQ', 'PRKCD', 'PTPA', 'SREBF1', 'IRS2', 'TRIB3', 'PYGM', 'PYGB', 'PYGL', 'FOXO1', 'PCK1', 'PCK2', 'G6PC', 'G6PC2', 'G6PC3', 'SLC2A2', 'NR1H3', 'NR1H2', 'PPARGC1B', 'MLX', 'MLXIP', 'MLXIPL', 'CPT1A', 'PRKCE', 'PPARGC1A', 'PPARA', 'PDPK1', 'PTEN', 'OGA', 'OGT', 'SLC2A1', 'GFPT1', 'GFPT2', 'CRTC2', 'CREB1', 'CREB3', 'CREB3L1', 'CREB3L

In [7]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
"""figure out highest match for each at each time """

keys = ['ND', 'HFD8', 'HFD14']

lfcT = 0
sensT = 0.1
tp = 15


res = []

for key in keys:
    
    # process ligands
    lf = lig[lig['key'] == key]
    lf = lf[lf['gene'].isin(ldf['ligand'].unique())]
    lf = lf[lf['true_positive'] > tp]
    lf = lf[lf['LFC'] > lfcT]
    lf = lf[lf['sensitivity'] > sensT].reset_index(drop=True)
    lf['nCells'] = lf['true_positive'] + lf['false_negative']
    
    g = lf.groupby('cellType').agg(
        n = ('gene', 'count'),
        nUnique = ('gene', 'nunique'),
        meanSensitivity = ('sensitivity', 'mean'),
        meanF1 = ('F1', 'mean'),
        meanLFC = ('LFC', 'mean'),
        nCells = ('nCells', 'first'),
    ).reset_index(drop=False)
    
    g['type'] = 'ligand'
    g['key'] = key
    res.append(g)
    
    # process ligands
    rf = rec[rec['key'] == key]
    rf = rf[rf['gene'].isin(ldf['receptor'].unique())]
    rf = rf[rf['true_positive'] > tp]
    rf = rf[rf['LFC'] > lfcT]
    rf = rf[rf['sensitivity'] > sensT].reset_index(drop=True)
    rf['nCells'] = rf['true_positive'] + rf['false_negative']
    
    g = rf.groupby('cellType').agg(
        n = ('gene', 'count'),
        nUnique = ('gene', 'nunique'),
        meanSensitivity = ('sensitivity', 'mean'),
        meanF1 = ('F1', 'mean'),
        meanLFC = ('LFC', 'mean'),
        nCells = ('nCells', 'first'),
    ).reset_index(drop=False)
    
    g['type'] = 'receptor'
    g['key'] = key
    res.append(g)
    
res = pd.concat(res, ignore_index=True)
res['LRpCell'] = res['n'] / res['nCells']
res.head()

In [ ]:

plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.facecolor'] = "w"
plt.rcParams['figure.figsize'] = 15, 4.5

fig, axs = plt.subplots(1, 3, sharey=True)
axs = axs.ravel()

cTypes = sorted(res['cellType'].unique())

for i, key in enumerate(keys):
    
    pdf = res[res['key'] == key]
    
    # ensure every cell type is in the plot
    stub = pd.DataFrame({'cellType' : res['cellType'].unique().copy()})    
    pdf = pd.merge(stub['cellType'], 
                   pdf, 
                   how='left', 
                   left_on='cellType', 
                   right_on='cellType')
    
    pdf['LRpCell'] = pdf['LRpCell'].fillna(0)
    pdf = pdf.sort_values(by='cellType')
    
    sns.barplot(data=pdf,
                y='cellType',
                x='LRpCell',
                hue='type',
                ec='k',
                zorder=2,
                ax=axs[i])

    axs[i].legend_.remove()
    if i == 2:
        axs[i].legend(loc='upper right')
    axs[i].set_title(key)
    axs[i].grid(True, zorder=0)
    axs[i].set_ylabel("")
    axs[i].set_xlabel(f"LR per cell expressed")
    axs[i].xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    # break
    
plt.tight_layout()
plt.suptitle(pathname, y=1.05)
print(pathname, pathId)